# Text Generation models

## Chat Completion API

In [ ]:
# !pip install -q openai
# !pip install -q python-dotenv

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#### 메시지와 역할 (Messages and Roles)

Chat Completions API에서는 모델에 지침을 제공하는 **메시지 배열(array of messages)** 을 통해 프롬프트를 생성합니다.  
각 메시지는 **다른 역할(Role)** 을 가질 수 있으며, 이는 모델이 입력을 해석하는 방식에 영향을 미칩니다.



## **역할(Role) 설명**

| **Role**      | **설명**                                                                                              | **사용 예제**                                                                                                                                                                   |
|---------------|------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **user**      | 모델에게 특정 출력을 요청하는 메시지. ChatGPT에서 사용자가 입력하는 일반적인 메시지와 동일합니다.           | `"재귀함수에 대한 시를 작성해줘."`                                                                                                                              |
| **developer** | 사용자 메시지보다 우선적으로 적용되는 모델 지침. 이전에는 "system prompt"로 불렸습니다.                      | `"당신은 미국 남동부 지역 사투리를 사용하는 프로그래밍 질문 도우미입니다. 이제 모든 응답은 남부 스타일의 말투로 작성되어야 합니다."`                                                                                 |
| **assistant** | 모델이 생성한 메시지로, 이전 요청에서 생성된 응답을 포함할 수 있습니다.                                     |  `"똑똑! 누구세요? 나는 OpenAI야! OpenAI 누구?"`                                                                            |

---

## **메시지 역할의 중요성**
- **더 나은 응답 생성**: 역할을 적절히 사용하면 **더 정교하고 원하는 방식의 결과**를 얻을 수 있습니다.
- **계층적 지침 전달**: `developer` 역할을 사용하여 모델의 동작과 톤을 정의할 수 있습니다.
- **비결정적(Dynamic)**: 각 역할의 효과는 상황에 따라 달라질 수 있으므로, 여러 접근 방식을 시도해보는 것이 좋습니다.

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

response = client.chat.completions.create(
  model=Model,
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "2020년 코리안 시리즈를 우승한 야구팀이 어디?"},
    {"role": "assistant", "content": "2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다."},
    {"role": "user", "content": "마지막 게임이 어디에서 열렸어?"}
  ]
)

In [3]:
response.to_dict()

{'id': 'chatcmpl-AubCVpSBd7nzQS61yyrfJI7A9ia6c',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '2020년 한국시리즈 7차전은 창원NC파크에서 열렸습니다. NC 다이노스가 홈 구장에서 우승을 차지했습니다.',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1738052731,
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_72ed7ab54c',
 'usage': {'completion_tokens': 38,
  'prompt_tokens': 81,
  'total_tokens': 119,
  'completion_tokens_details': {'audio_tokens': 0,
   'reasoning_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}

In [4]:
print(response.choices[0].message.content)

2020년 한국시리즈 7차전은 창원NC파크에서 열렸습니다. NC 다이노스가 홈 구장에서 우승을 차지했습니다.


- 모든 응답에는 'finish_reason'이 포함되며, 'finish_reason'의 가능한 값은 다음과 같습니다:
    - stop: API가 완전한 메시지를 반환하거나, stop 매개변수를 통해 제공된 중단 시퀀스 중 하나에 의해 메시지가 종료됨
    - length: max_tokens 매개변수 또는 토큰 제한으로 인해 완전하지 않은 모델 출력
    - function_call: 모델이 함수를 호출하기로 결정함

In [6]:
print(response.choices[0].finish_reason)

stop


### 구조화된 출력 (Structured Outputs)

- JSON 스키마를 준수하는 응답을 보장합니다.  

- JSON은 애플리케이션 간 데이터 교환을 위한 전 세계적으로 널리 사용되는 포맷 중 하나입니다.  

- 구조화된 출력(Structured Outputs)은 모델이 항상 제공된 [JSON 스키마](https://json-schema.org/overview/what-is-jsonschema)를 준수하도록 보장하는 기능입니다. 이를 통해 모델이 필요한 키를 누락하거나, 잘못된 열거형(enum) 값을 생성하는 걱정을 덜 수 있습니다.

**구조화된 출력의 주요 장점**

1. **신뢰할 수 있는 타입 안정성**: 잘못된 형식의 응답을 검증하거나 재요청할 필요가 없습니다.  
2. **명시적인 거부**: 안전성 기반의 모델 거부를 프로그래밍 방식으로 감지할 수 있습니다.  
3. **간단한 프롬프트**: 일관된 형식을 달성하기 위해 강한 표현의 프롬프트가 필요하지 않습니다.  

**SDK와 통합**  

REST API뿐만 아니라, OpenAI의 [Python](https://github.com/openai/openai-python/blob/main/helpers.md#structured-outputs-parsing-helpers) SDK에서도 [Pydantic](https://docs.pydantic.dev/latest/)를 사용하여 객체 스키마를 정의하기 쉽게 지원합니다.

In [9]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "developer", "content": "이벤트 정보를 추출하세요. 한국어로 추출하세요"},
        {"role": "user", "content": "앨리스와 밥이 금요일에 과학 박람회에 갈 예정입니다."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [10]:
event

CalendarEvent(name='과학 박람회', date='금요일', participants=['앨리스', '밥'])

## Token 관리
- "ChatGPT is great!“ 는 6개의 토큰으로 인코딩됩니다 $\rightarrow$ ["Chat", "G", "PT", " is", " great", "!"]
- 예를 들어, API 호출이 메시지 입력에서 10개의 토큰을 사용하고 메시지 출력에서 ​​20개의 토큰을 받은 경우 30개의 토큰에 대한 요금이 청구됩니다.


In [20]:
import tiktoken

encoding = tiktoken.encoding_for_model(Model)

messages=[
    {"role": "developer", "content": "당신은 도움이 되는 어시스턴트입니다."},
    {"role": "user", "content": "2020년 코리안 시리즈를 우승한 야구팀이 어디?"},
    {"role": "assistant", "content": "2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다."},
    {"role": "user", "content": "마지막 게임이 어디에서 열렸어? 한국어로 답해줘"}
  ]

num_tokens = 0
for message in messages:
  num_tokens += 4  #모든 메시지는 다음 형식을 따릅니다: `<im_start>{role/name}\n{content}<im_end>\n`
  for key, value in message.items():
      num_tokens += len(encoding.encode(value))
      print(key, value) # role, role value 
      print(encoding.encode(value))  # encoding value
  print()
    
print(num_tokens)

role developer
[77944]
content 당신은 도움이 되는 어시스턴트입니다.
[19388, 11753, 4740, 159807, 107081, 21252, 5637, 5648, 74404, 7984, 27001, 13]

role user
[1428]
content 2020년 코리안 시리즈를 우승한 야구팀이 어디?
[1323, 15, 12622, 43824, 4960, 14307, 12468, 186042, 4831, 26090, 42643, 3748, 109887, 10997, 91214, 2186, 154074, 30]

role assistant
[173781]
content 2020년 한국시리즈(Korean Series) 우승 야구팀은 NC 다이노스(NC Dinos)입니다.
[1323, 15, 12622, 52971, 5637, 186042, 28797, 99693, 14932, 8, 26090, 42643, 109887, 10997, 91214, 4740, 28895, 14367, 2186, 21464, 5648, 10564, 34, 415, 8200, 8, 27001, 13]

role user
[1428]
content 마지막 게임이 어디에서 열렸어? 한국어로 답해줘
[11630, 118016, 55377, 2186, 154074, 11440, 49496, 76892, 5959, 30, 52971, 5959, 3710, 107393, 5650, 153545]

94


## 재현 가능한 출력 
- SEED 매개 변수 사용

In [21]:
from IPython.display import display, HTML

MODEL = "gpt-4o-mini"
SEED = 123

topic = "화성으로의 여행"
system_message = "당신은 짧은 이야기를 창작하는 소설가 입니다."
user_request = f"{topic}에 관한 짧은 이야기를 한국어로 생성하세요."

try:
    messages = [
        {"role": "developer", "content": system_message},
        {"role": "user", "content": user_request},
    ]

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        seed=SEED,
        max_tokens=1000,
        temperature=1.2,
    )

    response_content = response.choices[0].message.content
    system_fingerprint = response.system_fingerprint  # 특정 API 응답을 식별하고 추적하는 데 도움 되는 고유 식별자
    completion_tokens = (
        response.usage.total_tokens - response.usage.prompt_tokens
    )
    finish_reason = response.choices[0].finish_reason

    table = f"""
    <table>
    <tr><th style="text-align: left;">Response</th><td style="text-align: left;">{response_content}</td></tr>
    <tr><th style="text-align: left;">System Fingerprint</th><td style="text-align: left;">{system_fingerprint}</td></tr>
    <tr><th style="text-align: left;">Number of completion tokens</th><td style="text-align: left;">{completion_tokens}</td></tr>
    <tr><th style="text-align: left;">finish reason</th><td style="text-align: left;">{finish_reason}</td></tr>
    </table>
    """
    display(HTML(table))
except Exception as e:
    print(f"An error occurred: {e}")

Response,"화성으로의 여행은 인류의 꿈이었다. 25세기, 기술이 발전한 이 시대에 마침내 그 꿈을 현실로 만들 기회가 찾아왔다. NASA는 최초의 유인 화성 탐사 프로젝트를 시작했고, 다양한 배경을 가진 우주 탐험가들이 모집되었다. 그 중 가장 젊은 탐험가는 불과 20살의 리안이었다. 리안은 어릴 적부터 별과 우주에 대한 호기심이 남다른 아이였다. 그러던 어느 날, 화성에서의 삶이 그려진 책 한 권을 통해 화성을 동경하게 되었고, 망원경을 통해 바라보는 붉은 행성의 모습에 매료됐다. 마침내 드디어 화성 여행이 시작되었고, 리안은 동료들과 함께 '서프라이즈01'호에 탑승했다. 우주선 안에서 친구, 에이미와 함께 거대한 유리창에 비친 익숙한 지구를 바라보며 둘은 다짐했다. “우리 집에 무사히 돌아갈 수 있을까?” 서로의 손을 맞잡으며 화성에 착륙할 준비를 했다. 여행 도중, 무중력 상태에서의 목욕은 비눗방울이 자유롭게 떠 다니는 마법 같은 순간이었고, 뒷 선실에서는 첨단 장비를 만지며 꿈에 그리던 화성과 가까워짐을 느꼈다. 하지만 화성의 대기는 무척 차가웠고, 지구와는 전혀 다른 인상적이고도 쓸쓸한 곳이었다. 붉은 먼지와 돌들이 널려 있었고, '서프라이즈01'호의 인젝션 프로펠러 소리에 그 장면은 더욱 극적이었다. 착륙 후 리안과 동료들은 우주복을 입고 탐사에 나섰다. 마르스의 산과 계곡을 탐험하며 고대의 미소 선들의 흔적을 발견하고, 자생하는 식물체도 체험하며 화성 문명 탐구에 몰두했다. 리안은 우주선으로부터 멀리 떨어진 곳에서 고대 유적 같은 곳을 발견했고, 식민지의 잔재와 그들이 남깁니다. 이른 아침, 스며드는 햇빛에 감아졌던 회의감이 가시었다. 여행의 마지막 날, 리안은 갈라진 대지 위에 서서 붉은 태양이 지평선 너머로 사라지는 장면을 바라보았다. “우리가 꿈꿔온 한 점, 발견하게 되어 기쁘다”며 눈을 감았다. 내내 느낀 언어로도 표현할 수 없는 기쁨, 그리고 그것을 함께한 모든 동료들에게 감사가 깃들던 순간이었다. 리안과 동료들은 안전하게 지구로 돌아왔고, 그들이 촬영한 화성의 사진들은 세상을 바꿀 이야기가 되었다. 문명은 그들의 여행에서 영감을 받아 화성 탐사의 이정표가 형성되었고, 언제가 우리가 부른 이름 ‘화성’은 이제 새로운 집이라는 꿈이 현실로 다가오는 날이 기다려졌다. 지구를 향해 나아가는 우주선, 그들은 다짐했다. “앞으로도 새로운 세상을 보러 나가자.”"
System Fingerprint,fp_72ed7ab54c
Number of completion tokens,750
finish reason,stop


In [22]:
response.to_dict()

{'id': 'chatcmpl-AubX9D30WX4Z9tcVXOtKx8W13An8d',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': "화성으로의 여행은 인류의 꿈이었다. 25세기, 기술이 발전한 이 시대에 마침내 그 꿈을 현실로 만들 기회가 찾아왔다. NASA는 최초의 유인 화성 탐사 프로젝트를 시작했고, 다양한 배경을 가진 우주 탐험가들이 모집되었다. 그 중 가장 젊은 탐험가는 불과 20살의 리안이었다.\n\n리안은 어릴 적부터 별과 우주에 대한 호기심이 남다른 아이였다. 그러던 어느 날, 화성에서의 삶이 그려진 책 한 권을 통해 화성을 동경하게 되었고, 망원경을 통해 바라보는 붉은 행성의 모습에 매료됐다.\n\n마침내 드디어 화성 여행이 시작되었고, 리안은 동료들과 함께 '서프라이즈01'호에 탑승했다. 우주선 안에서 친구, 에이미와 함께 거대한 유리창에 비친 익숙한 지구를 바라보며 둘은 다짐했다. “우리 집에 무사히 돌아갈 수 있을까?” 서로의 손을 맞잡으며 화성에 착륙할 준비를 했다.\n\n여행 도중, 무중력 상태에서의 목욕은 비눗방울이 자유롭게 떠 다니는 마법 같은 순간이었고, 뒷 선실에서는 첨단 장비를 만지며 꿈에 그리던 화성과 가까워짐을 느꼈다. 하지만 화성의 대기는 무척 차가웠고, 지구와는 전혀 다른 인상적이고도 쓸쓸한 곳이었다. 붉은 먼지와 돌들이 널려 있었고, '서프라이즈01'호의 인젝션 프로펠러 소리에 그 장면은 더욱 극적이었다.\n\n착륙 후 리안과 동료들은 우주복을 입고 탐사에 나섰다. 마르스의 산과 계곡을 탐험하며 고대의 미소 선들의 흔적을 발견하고, 자생하는 식물체도 체험하며 화성 문명 탐구에 몰두했다. 리안은 우주선으로부터 멀리 떨어진 곳에서 고대 유적 같은 곳을 발견했고, 식민지의 잔재와 그들이 남깁니다. 이른 아침, 스며드는 햇빛에 감아졌던 회의감이 가시었다.\n\n여행의 마지막 날, 리안은 갈라

### 수학문제 풀이

In [23]:
completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "developer", "content": "당신은 도움이 되는 조수입니다. 내 수학 숙제를 도와주세요!"}, # <-- 모델에 컨텍스트를 제공하는 시스템 메시지입니다.
    {"role": "user", "content": "안녕하세요! 2+2를 풀 수 있나요?"}  # <-- 모델이 응답을 생성할 사용자 메시지입니다.
  ]
)

print("Assistant: " + completion.choices[0].message.content)

Assistant: 안녕하세요! 물론입니다. 2+2는 4입니다. 더 도와드릴 것이 있나요?
